In [2]:
# coding:gbk
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# 载入数据集
mnist = input_data.read_data_sets("MINST_data/", one_hot=True)


Extracting MINST_data/train-images-idx3-ubyte.gz
Extracting MINST_data/train-labels-idx1-ubyte.gz
Extracting MINST_data/t10k-images-idx3-ubyte.gz
Extracting MINST_data/t10k-labels-idx1-ubyte.gz


In [3]:
# 每个批次的大小
batch_size = 100

# 计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

# 定义input的命名空间
with tf.name_scope("input"):
    # 定义两个placeholder
    x = tf.placeholder(tf.float32, [None, 784], name="x-input")
    y = tf.placeholder(tf.float32, [None, 10], name="y-input")


# 定义命名空间
with tf.name_scope("layer"):
    # 创建一个简单的神经网络
    # 定义权值的命名空间
    with tf.name_scope("wights"):
        w = tf.Variable(tf.zeros([784, 10]), name="W")
    # 定义偏置值的命名空间
    with tf.name_scope("biases"):
        b = tf.Variable(tf.zeros(10), name="b")
    # 定义wx_plus_b命名空间
    with tf.name_scope("wx_plus_b"):
        wx_plus_b = tf.matmul(x, w) + b
# 预测值
# prediction = tf.nn.softmax(tf.matmul(x, w)+b)
    # 定义softmax命名空间
    with tf.name_scope("softmax"):
        prediction = tf.nn.softmax(wx_plus_b)

# 二次代价函数
# loss = tf.reduce_mean(tf.square(y-prediction))
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))
    with tf.name_scope('train'):
        # 使用梯度下降法
        train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

# 初始化变量
init = tf.global_variables_initializer()

# 函数equal比较两个参数的大小，一样为True， 不一样为False
# 找出真实值的最大值标签和预测值的最大值标签
# 结果存放在一个布尔值列表中
# argmax函数返回一维张量中最大值所在的位置
# 真实值与预测值的对比
with tf.name_scope("accuracy"):
    with tf.name_scope("correct_prediction"):
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1))
    with tf.name_scope("accuracy"):
        # 求准确率
        # cast函数将correct_prediction由布尔类型转换为浮点型
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# 构建会话
with tf.Session() as sess:
    sess.run(init)
    # "logs/":路径，sess.graph：存放的文件
    # writer = tf.summary.FileWriter("logS/", sess.graph)
    writer = tf.summary.FileWriter("los/", sess.graph)
    # 所有图片训练1次
    for epoch in range(1):
        # 训练n_batch次
        for batch in range(n_batch):
            # 图片的的信心保存在batch_xs中，图片的标签保存在batch_ys中
            # 第一次是0-100，第二次是100-200，第三次是200-300，依次类推
            # 每一个批次是100
            # 获得批次的数据
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # 对获得的数据进行训练,直到把所有的图片训练一次
            sess.run(train_step, feed_dict={x: batch_xs, y : batch_ys})

        # 测试准确率
        acc = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels})
        # 打印训练次数和每一次的准确率
        print("Iter(训练次数):" + str(epoch) + " ,Testing Accuracy(正确率):" + str(acc))


W0413 16:40:37.235875  5516 deprecation.py:323] From <ipython-input-3-c15f8840c474>:35: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



Iter(训练次数):0 ,Testing Accuracy(正确率):0.825


In [7]:
# coding:gbk
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# 载入数据集
mnist = input_data.read_data_sets("MINST_data", one_hot=True)

# 每个批次的大小
batch_size = 100

# 计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

# 参数概要
# 定义一个计算各类值的函数
def variable_summaries(var):
    with tf.name_scope("summaries"):
        mean =  tf.reduce_mean(var)
        # tf.summary.scalar参数:第一个参数为名字，第二个参数为所求的值
        tf.summary.scalar("mean", mean)  # 平均值
        with tf.name_scope("stddev"):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
            tf.summary.scalar("stddev", stddev)  # 标准差
            tf.summary.scalar("max", tf.reduce_max(var))  # 最大值
            tf.summary.scalar("min", tf.reduce_min(var))  # 最小值
            tf.summary.histogram("histogram", var)   # 直方图

# 定义input的命名空间
with tf.name_scope("input"):
    # 定义两个placeholder
    x = tf.placeholder(tf.float32, [None, 784], name="x-input")
    y = tf.placeholder(tf.float32, [None, 10], name="y-input")


# 定义命名空间
with tf.name_scope("layer"):
    # 创建一个简单的神经网络
    # 定义权值的命名空间
    with tf.name_scope("wights"):
        w = tf.Variable(tf.zeros([784, 10]), name="W")
        # 分析权值的变化
        variable_summaries(w)
    # 定义偏置值的命名空间
    with tf.name_scope("biases"):
        b = tf.Variable(tf.zeros(10), name="b")
        # 分析偏置值的变化
        variable_summaries(b)
    # 定义wx_plus_b命名空间
    with tf.name_scope("wx_plus_b"):
        wx_plus_b = tf.matmul(x, w) + b
# 预测值
# prediction = tf.nn.softmax(tf.matmul(x, w)+b)
    # 定义softmax命名空间
    with tf.name_scope("softmax"):
        prediction = tf.nn.softmax(wx_plus_b)

# 二次代价函数
# loss = tf.reduce_mean(tf.square(y-prediction))
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))
    tf.summary.scalar("loss", loss)
    with tf.name_scope('train'):
        # 使用梯度下降法
        train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

# 初始化变量
init = tf.global_variables_initializer()

# 函数equal比较两个参数的大小，一样为True， 不一样为False
# 找出真实值的最大值标签和预测值的最大值标签
# 结果存放在一个布尔值列表中
# argmax函数返回一维张量中最大值所在的位置
# 真实值与预测值的对比
with tf.name_scope("accuracy"):
    with tf.name_scope("correct_prediction"):
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1))
    with tf.name_scope("accuracy"):
        # 求准确率
        # cast函数将correct_prediction由布尔类型转换为浮点型
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        # 记录准确率的变化
        tf.summary.scalar("accuracy", accuracy)

# 合并所有的summary
merged = tf.summary.merge_all()

# 构建会话
with tf.Session() as sess:
    sess.run(init)
    # "logs/":路径，sess.graph：存放的文件
    # writer = tf.summary.FileWriter("logS/", sess.graph)
    writer = tf.summary.FileWriter("los/", sess.graph)
    # 所有图片训练50次
    for epoch in range(51):
        # 训练n_batch次
        for batch in range(n_batch):
            # 图片的的信心保存在batch_xs中，图片的标签保存在batch_ys中
            # 第一次是0-100，第二次是100-200，第三次是200-300，依次类推
            # 每一个批次是100
            # 获得批次的数据
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # 对获得的数据进行训练,直到把所有的图片训练一次
            # [merged,train_step]训练时同时统计权值、偏置值、loss的变化
            # 统计完成后有返回值，返回值为summary,_
            summary,_ = sess.run([merged, train_step], feed_dict={x: batch_xs, y : batch_ys})
        # 每执行一次epoch(次数),将summary写入指定的文件路径中
        writer.add_summary(summary, epoch)
        # 测试准确率
        acc = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels})
        # 打印训练次数和每一次的准确率
        print("Iter(训练次数):" + str(epoch) + " ,Testing Accuracy(正确率):" + str(acc))


Extracting MINST_data\train-images-idx3-ubyte.gz
Extracting MINST_data\train-labels-idx1-ubyte.gz
Extracting MINST_data\t10k-images-idx3-ubyte.gz
Extracting MINST_data\t10k-labels-idx1-ubyte.gz
Iter(训练次数):0 ,Testing Accuracy(正确率):0.8333
Iter(训练次数):1 ,Testing Accuracy(正确率):0.8941
Iter(训练次数):2 ,Testing Accuracy(正确率):0.9021
Iter(训练次数):3 ,Testing Accuracy(正确率):0.9067
Iter(训练次数):4 ,Testing Accuracy(正确率):0.9086
Iter(训练次数):5 ,Testing Accuracy(正确率):0.9103
Iter(训练次数):6 ,Testing Accuracy(正确率):0.912
Iter(训练次数):7 ,Testing Accuracy(正确率):0.9144
Iter(训练次数):8 ,Testing Accuracy(正确率):0.9152
Iter(训练次数):9 ,Testing Accuracy(正确率):0.9169
Iter(训练次数):10 ,Testing Accuracy(正确率):0.9168
Iter(训练次数):11 ,Testing Accuracy(正确率):0.9194
Iter(训练次数):12 ,Testing Accuracy(正确率):0.9183
Iter(训练次数):13 ,Testing Accuracy(正确率):0.9196
Iter(训练次数):14 ,Testing Accuracy(正确率):0.9203
Iter(训练次数):15 ,Testing Accuracy(正确率):0.9205
Iter(训练次数):16 ,Testing Accuracy(正确率):0.9215
Iter(训练次数):17 ,Testing Accuracy(正确率):0.9209
Iter(训练次数):18 ,Testing Ac

In [ ]:
# coding:gbk
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector


# 载入数据集
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

# 每个批次的大小
# batch_size = 100
# 运行次数
max_steps = 1001

# 图片数量
image_num = 3000

# 文件路径
DIR = "C:\\Users\\20622\\Tensorflow深度学习\\"
# 计算一共有多少个批次
# n_batch = mnist.train.num_examples // batch_size

# 定义会话
sess = tf.Session()

# 载入图片
embedding = tf.Variable(tf.stack(mnist.test.images[:image_num]), trainable=False, name="embedding")

# 参数概要
# 定义一个计算各类值的函数
def variable_summaries(var):
    with tf.name_scope("summaries"):
        mean =  tf.reduce_mean(var)
        # tf.summary.scalar参数:第一个参数为名字，第二个参数为所求的值
        tf.summary.scalar("mean", mean)  # 平均值
        with tf.name_scope("stddev"):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
            tf.summary.scalar("stddev", stddev)  # 标准差
            tf.summary.scalar("max", tf.reduce_max(var))  # 最大值
            tf.summary.scalar("min", tf.reduce_min(var))  # 最小值
            tf.summary.histogram("histogram", var)   # 直方图

# 定义input的命名空间
with tf.name_scope("input"):
    # 定义两个placeholder
    x = tf.placeholder(tf.float32, [None, 784], name="x-input")
    y = tf.placeholder(tf.float32, [None, 10], name="y-input")

# 显示图片
with tf.name_scope("input_reshape"):
    # 将x的形状转化为[-1, 28, 28, 1]的形状，-1代表不确定值，28行28列， 维度为1
    image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
    # 10表示10张图片
    tf.summary.image("input", image_shaped_input, 10)

# 定义命名空间
with tf.name_scope("layer"):
    # 创建一个简单的神经网络
    # 定义权值的命名空间
    with tf.name_scope("wights"):
        w = tf.Variable(tf.zeros([784, 10]), name="W")
        # 分析权值的变化
        variable_summaries(w)
    # 定义偏置值的命名空间
    with tf.name_scope("biases"):
        b = tf.Variable(tf.zeros(10), name="b")
        # 分析偏置值的变化
        variable_summaries(b)
    # 定义wx_plus_b命名空间
    with tf.name_scope("wx_plus_b"):
        wx_plus_b = tf.matmul(x, w) + b
# 预测值
# prediction = tf.nn.softmax(tf.matmul(x, w)+b)
    # 定义softmax命名空间
    with tf.name_scope("softmax"):
        prediction = tf.nn.softmax(wx_plus_b)

# 二次代价函数
# loss = tf.reduce_mean(tf.square(y-prediction))
with tf.name_scope('loss'):
    # 交叉熵代价函数
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))
    tf.summary.scalar("loss", loss)
    with tf.name_scope('train'):
        # 使用梯度下降法
        train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

# 初始化变量
init = tf.global_variables_initializer()
sess.run(init)

# 函数equal比较两个参数的大小，一样为True， 不一样为False
# 找出真实值的最大值标签和预测值的最大值标签
# 结果存放在一个布尔值列表中
# argmax函数返回一维张量中最大值所在的位置
# 真实值与预测值的对比
with tf.name_scope("accuracy"):
    with tf.name_scope("correct_prediction"):
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1))
    with tf.name_scope("accuracy"):
        # 求准确率
        # cast函数将correct_prediction由布尔类型转换为浮点型
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        # 记录准确率的变化
        tf.summary.scalar("accuracy", accuracy)

# 产生metadata文件
# 如果有这个文件， 就会删除这个对应的文件
if tf.gfile.Exists(DIR+"projector/projector/metadata.tsv"):
    tf.gfile.DeleteRecursively(DIR+"projector/projector/metadata.tsv")
    # tf.gfile.remove(DIR + "projector/projector/metadata.tsv")

# 没有这个文件，就会生成对应的文件
with open(DIR + "projector/projector/metadata.tsv", "w") as f:
    # 求出标签最大值所在的位置
    labels = sess.run(tf.argmax(mnist.test.labels[:], 1))
    # 循环3000次
    for i in range(image_num):
        # 将每次循环的结果写入文件中
        f.write(str(labels[i]) + "\n")

# 合并所有的summary
merged = tf.summary.merge_all()

projector_writer = tf.summary.FileWriter(DIR+"projector/projector", sess.graph)

# 保存网络模型
saver = tf.train.Saver()
# 配置项
config = projector.ProjectorConfig()
embed = config.embeddings.add()

# embedding的名字赋给embed.tensor
embed.tensor_name = embedding.name

# 文件路径
embed.metadata_path = DIR + "projector/projector/metadata.tsv"

# 图片路径
embed.sprite.image_path = DIR +"projector/data/mnist_10k_sprite.png"

# 切分为28x28的像素
embed.sprite.single_image_dim.extend([28, 28])

# 进入可视化的工具
projector.visualize_embeddings(projector_writer, config)

for i in range(max_steps):
    # 每个批次100个样本
    batch_xs, batch_ys = mnist.train.next_batch(100)

    # 配置项
    run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    run_metadata = tf.RunMetadata()

    summary,_ = sess.run([merged, train_step], feed_dict={x:batch_xs, y:batch_ys}, options=run_options, run_metadata=run_metadata)
    projector_writer.add_run_metadata(run_metadata, 'step%03d' % i)
    projector_writer.add_summary(summary, i)

    # 每训练100次100次， 打印准确率
    if i % 100 == 0:
        acc = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels})
        print("Iter" + str(i) + ", Testing Accuracy= " + str(acc))

# 保存训练好的模型
saver.save(sess, DIR + 'projector/projector/a_model.ckpt', global_step=max_steps)
projector_writer.close()
sess.close()

#
# # 构建会话
# with tf.Session() as sess:
#     sess.run(init)
#     # "logs/":路径，sess.graph：存放的文件
#     # writer = tf.summary.FileWriter("logS/", sess.graph)
#     writer = tf.summary.FileWriter("los/", sess.graph)
#     # 所有图片训练50次
#     for epoch in range(51):
#         # 训练n_batch次
#         for batch in range(n_batch):
#             # 图片的的信心保存在batch_xs中，图片的标签保存在batch_ys中
#             # 第一次是0-100，第二次是100-200，第三次是200-300，依次类推
#             # 每一个批次是100
#             # 获得批次的数据
#             batch_xs, batch_ys = mnist.train.next_batch(batch_size)
#             # 对获得的数据进行训练,直到把所有的图片训练一次
#             # [merged,train_step]训练时同时统计权值、偏置值、loss的变化
#             # 统计完成后有返回值，返回值为summary,_
#             summary,_ = sess.run([merged, train_step], feed_dict={x: batch_xs, y : batch_ys})
#         # 每执行一次epoch(次数),将summary写入指定的文件路径中
#         writer.add_summary(summary, epoch)
#         # 测试准确率
#         acc = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels})
#         # 打印训练次数和每一次的准确率
#         print("Iter(训练次数):" + str(epoch) + " ,Testing Accuracy(正确率):" + str(acc))
